# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff3d4256dc0>
├── 'a' --> tensor([[-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585]])
└── 'x' --> <FastTreeValue 0x7ff3d4256910>
    └── 'c' --> tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                        [ 0.0497, -0.4454, -0.2778, -1.5699],
                        [ 0.1366,  0.3614,  0.3101,  1.6129]])

In [4]:
t.a

tensor([[-0.8246,  0.4348,  0.1000],
        [-0.6806, -0.9627, -0.7585]])

In [5]:
%timeit t.a

60.6 ns ± 0.0402 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff3d4256dc0>
├── 'a' --> tensor([[-2.7817e-01,  2.0620e-01, -1.8968e+00],
│                   [ 1.2979e-01,  7.4491e-01, -7.5679e-04]])
└── 'x' --> <FastTreeValue 0x7ff3d4256910>
    └── 'c' --> tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                        [ 0.0497, -0.4454, -0.2778, -1.5699],
                        [ 0.1366,  0.3614,  0.3101,  1.6129]])

In [7]:
%timeit t.a = new_value

59.1 ns ± 0.0386 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585]]),
    x: Batch(
           c: tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                      [ 0.0497, -0.4454, -0.2778, -1.5699],
                      [ 0.1366,  0.3614,  0.3101,  1.6129]]),
       ),
)

In [10]:
b.a

tensor([[-0.8246,  0.4348,  0.1000],
        [-0.6806, -0.9627, -0.7585]])

In [11]:
%timeit b.a

59.5 ns ± 0.0675 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7154,  0.1400,  1.7908],
               [ 0.4638,  0.0072, -0.1735]]),
    x: Batch(
           c: tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                      [ 0.0497, -0.4454, -0.2778, -1.5699],
                      [ 0.1366,  0.3614,  0.3101,  1.6129]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.239 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 5.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 58.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 602 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff31e5050a0>
├── 'a' --> tensor([[[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]],
│           
│                   [[-0.8246,  0.4348,  0.1000],
│                    [-0.6806, -0.9627, -0.7585]]])
└── 'x' --> <FastTreeValue 0x7ff31e505cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff325483ac0>
├── 'a' --> tensor([[-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585],
│                   [-0.8246,  0.4348,  0.1000],
│                   [-0.6806, -0.9627, -0.7585]])
└── 'x' --> <FastTreeValue 0x7ff31e4e5730>
    └── 'c' --> tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                        [ 0.0497, -0.4454, -0.2778, -1.5699],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 58.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 56.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]],
       
               [[-0.8246,  0.4348,  0.1000],
                [-0.6806, -0.9627, -0.7585]]]),
    x: Batch(
           c: tensor([[[ 1.9273, -1.8963,  0.1183, -0.1746],
                       [ 0.0497, -0.4454, -0.2778, -1.5699],
                       [ 0.1366,  0.3614,  0.3101,  1.6129]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585],
               [-0.8246,  0.4348,  0.1000],
               [-0.6806, -0.9627, -0.7585]]),
    x: Batch(
           c: tensor([[ 1.9273, -1.8963,  0.1183, -0.1746],
                      [ 0.0497, -0.4454, -0.2778, -1.5699],
                      [ 0.1366,  0.3614,  0.3101,  1.6129],
                      [ 1.9273, -1.8963,  0.1183, -0.1746],
                      [ 0.0497, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 6.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
